# Predicting Earnings of French People
***

***Piotr Skalski - 05.11.2017***

<img src='http://res.freestockphotos.biz/pictures/8/8163-euro-coins-and-bills-pv.jpg'>

## Table of Contents
***
* [1. Introduction](#introduction) <br>
* [2. Importing dataset and data preprocessing](#importing_dataset_and_data_preprocessing) <br>
   * [2.1. Importing essential libraries](#importing_essential_libraries) <br>
   * [2.2. Importing datasets](#importing_datasets) <br>
   * [2.3. Let's summarize the datasets](#lets_summarize_the_dataset) <br>
   * [2.4. Data preprocessing & feature engineering](#data_preprocessing) <br>
      * [2.4.1. Salary dataset preprocessing](#salary_dataset_preprocessing) <br>
      * [2.4.2. Geography dataset preprocessing](#geography_dataset_preprocessing) <br>
      * [2.4.3. Industry dataset preprocessing](#industry_dataset_preprocessing) <br>
      * [2.4.4. Merging datasets](#merging_datasets) <br>
      * [2.4.5. Splitting dataset into training and testset](#splitting_dataset_into_training_and_testset) <br>
* [3. Prediction](#prediction) <br>
   * [3.1. Linear Regression](#linear_regression) <br>
   * [3.2. Random Forest Regressor](#random_forest_regressor) <br>
   * [3.3. Evaluating the regression model](#evaluating_regression_model) <br>
***

## 1. Introduction
<a id="introduction"></a>

The purpose of this kernel is to create a model that allow for predicting earnings in France, based on some basic information about the employee and where he lives. The analysis carried out here is largely basedo on conclusions from <a href="https://www.kaggle.com/skalskip/how-big-is-french-industry-data-visualization">How big is French Industry? [Data Visualization]</a> [1]. To fully understand the decisions we'll make while creating a model, it is a good idea to read above notebook.

I also note that I am a novice in Machine Learning and Data Science. Thanks in advance for any suggestions and hints. Please share with me your ideas on the features I can use.

## 2. Importing dataset and data preprocessing
<a id="importing_dataset_and_data_preprocessing"></a>

### 2.1. Importing essential libraries
<a id="importing_essential_libraries"></a>

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.optimize import curve_fit
import seaborn as sns

from math import radians, cos, sin, asin, sqrt

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

### 2.2. Importing datasets
<a id="importing_datasets"></a>

In [ ]:
# Datasets
geography = pd.read_csv("../input/name_geographic_information.csv")
industry = pd.read_csv("../input/base_etablissement_par_tranche_effectif.csv")
salary = pd.read_csv("../input/net_salary_per_town_categories.csv")
population = pd.read_csv("../input/population.csv")
# Geojson for map creations
departments_map = gpd.read_file('../input/departements.geojson')

### 2.3. Let's summarize the datasets
<a id="lets_summarize_the_dataset"></a>

In [ ]:
geography.info()

In [ ]:
industry.info()

In [ ]:
salary.info()

In [ ]:
population.head(5)

In [ ]:
population.info()

### 2.4. Data preprocessing & feature engineering
<a id="data_preprocessing"></a>

<b>NOTE:</b> As you can see our dataset consists of four separate files, which store different types of information. Unfortunately, the information in these files is often contaminated or incomplete. At this point our task is to clean up the data and merge four files into one consistent DataFrame which can be used to train the model.

### 2.4.1. Salary dataset preprocessing
<a id="salary_dataset_preprocessing"></a>

<b>NOTE:</b> Salary dataset contains key information on the level of earnings of French citizens according to their place of residence, gender, occupation and age. Due to the very unfavorable data structure we will be forced to create new DataFrame and fill it based on properly processed data from salary dataset.

In [ ]:
salary = salary[salary["CODGEO"].apply(lambda x: str(x).isdigit())]
salary["CODGEO"] = salary["CODGEO"].astype(int)

In [ ]:
salary.describe()

<b>NOTE:</b> Let's save a copy of salary dataset, in case we will need it later. 

In [ ]:
salary_copy = salary.copy()

<b>NOTE:</b> As we have seen in [1] there is a strong connection between sex, age and occupation, and the amount of money earned. Unfortunately, due to the way dataset was build, we can not use all of these attributes at the same time. Firstly, let's look at how the pay varies depending on gender and age.

In [ ]:
age = ["18-25 years old", "26-50 years old", ">50 years old"]
woman_age = ["SNHMF1814", "SNHMF2614", "SNHMF5014"]
woman_salary_age = salary[woman_age].mean().tolist()
man_age = ["SNHMH1814", "SNHMH2614", "SNHMH5014"]
man_salary_age = salary[man_age].mean().tolist()

dif_in_prc_age = []
for w, m in zip(woman_salary_age, man_salary_age):
    dif_in_prc_age.append(round(abs(w-m)/m * 100, 2))
    
trace1 = go.Bar(
    x = age,
    y = woman_salary_age,
    name='Women',
    marker=dict(
        color='rgba(55, 128, 191, 0.7)',
        line=dict(
            color='rgba(55, 128, 191, 1.0)',
            width=2,
        )
    )
)
trace2 = go.Bar(
    x = age,
    y = man_salary_age,
    name='Men',
    marker=dict(
        color='rgba(219, 64, 82, 0.7)',
        line=dict(
            color='rgba(219, 64, 82, 1.0)',
            width=2,
        )
    )
)

trace3 = go.Scatter(
    x = age,
    y = dif_in_prc_age,
    name='Earnings difference',
    mode = 'lines+markers',
    yaxis='y2'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='group',
    title = 'Age and sex are',
    width=850,
    height=500,
    paper_bgcolor='rgb(244, 238, 225)',
    plot_bgcolor='rgb(244, 238, 225)',
    yaxis = dict(
        title= 'Average earnings [€/hour]',
        anchor = 'x',
        rangemode='tozero'
    ),
    xaxis = dict(title= 'Age'),
    
    yaxis2=dict(
        title='Earnings difference',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right',
        anchor = 'x',
        rangemode = 'tozero',
        dtick = 7.3
    ),
    #legend=dict(x=-.1, y=1.2)
    legend=dict(x=0.72, y=0.05)
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [ ]:
positions = ["Executive", "Middle manager", "Employee", "Worker"]
woman_positions = ["SNHMFC14", "SNHMFP14", "SNHMFE14", "SNHMFO14"]
woman_salary_positions = salary[woman_positions].mean().tolist()
man_positions = ["SNHMHC14", "SNHMHP14", "SNHMHE14", "SNHMHO14"]
man_salary_positions = salary[man_positions].mean().tolist()

dif_in_prc = []
for w, m in zip(woman_salary_positions, man_salary_positions):
    dif_in_prc.append(round(abs(w-m)/m * 100, 2))

trace1 = go.Bar(
    x = positions,
    y = woman_salary_positions,
    name='Women',
    marker=dict(
        color='rgba(55, 128, 191, 0.7)',
        line=dict(
            color='rgba(55, 128, 191, 1.0)',
            width=2,
        )
    )
)
trace2 = go.Bar(
    x = positions,
    y = man_salary_positions,
    name='Men',
    marker=dict(
        color='rgba(219, 64, 82, 0.7)',
        line=dict(
            color='rgba(219, 64, 82, 1.0)',
            width=2,
        )
    )
)

trace3 = go.Scatter(
    x = positions,
    y = dif_in_prc,
    name='Earnings difference',
    mode = 'lines+markers',
    yaxis='y2'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='group',
    title = 'Stereotype is real',
    width=850,
    height=500,
    paper_bgcolor='rgb(244, 238, 225)',
    plot_bgcolor='rgb(244, 238, 225)',
    yaxis = dict(
        title= 'Average earnings [€/hour]',
        anchor = 'x',
        rangemode='tozero'
    ),
    xaxis = dict(title= 'Position'),
    
    yaxis2=dict(
        title='Earnings difference',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right',
        anchor = 'x',
        rangemode = 'tozero',
        dtick = 8
    ),
    #legend=dict(x=-.1, y=1.2)
    legend=dict(x=0.05, y=0.05)
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

<b>NOTE:</b> Let's create new DataFrame that will separate information for different age groups. DataFrame will have four columns: "CODGEO", "SEX" (1 - for woman, 2 - for man), "AGE" (1 - for age between 18-25 years, 2 - for age between 26-50 years, 3 - for age over 50 years) and "WAGE" that will hold mean net salary per hour.

In [ ]:
row_fields = {}
row_fields["SNHMFC14"] = {"SEX": 1, "POSITION": 4}
row_fields["SNHMFP14"] = {"SEX": 1, "POSITION": 3}
row_fields["SNHMFE14"] = {"SEX": 1, "POSITION": 2}
row_fields["SNHMFO14"] = {"SEX": 1, "POSITION": 1}
row_fields["SNHMHC14"] = {"SEX": 2, "POSITION": 4}
row_fields["SNHMHP14"] = {"SEX": 2, "POSITION": 3}
row_fields["SNHMHE14"] = {"SEX": 2, "POSITION": 2}
row_fields["SNHMHO14"] = {"SEX": 2, "POSITION": 1}

reformatted_salary = []
for index, row in salary.iterrows():
    for key, value in row_fields.items(): 
        row_dict = {}
        row_dict["CODGEO"] = row["CODGEO"]
        row_dict["SEX"] = value["SEX"]
        row_dict["POSITION"] = value["POSITION"]
        row_dict["WAGE"] = row[key]
        reformatted_salary.append(row_dict)
        
reformatted_salary = pd.DataFrame(reformatted_salary)  

<b>NOTE: </b> Let's check if all the operations are done correctly.

In [ ]:
reformatted_salary.info()

### 2.4.2. Geography dataset preprocessing
<a id="geography_dataset_preprocessing"></a>

<b>NOTE:</b> During our analysis in [1] we have noticed that the average earning in districts close to Paris is definitely higher than in the rest of the country. Similarly, in cities located in the vicinity of provincial towns, the average income is higher than in the rest of the region. Let's create two new features: PARIS_CLOSE with values equal 1 or 0, depending on whether the place is closer or farther than 30 km from the center of Paris; MAJOR_CITY_DISTANCE: which will represent the distance in kilometers from the provincial town. 

<b>NOTE:</b> Unfortunately, some of the data in geography DataFrame is incorrect or incomplete. First we will focus on longitude column. We will convert commas to dots to create a correct number [#1]. Then we will delete the incorrect entries [#2] and empty fields [#3]. Finally, we will switch the column type to float [#4].

In [ ]:
# 1
geography["longitude"] = geography["longitude"].apply(lambda x: str(x).replace(',','.'))
# 2
mask = geography["longitude"] == '-'
geography.drop(geography[mask].index, inplace=True)
# 3
geography.dropna(subset = ["longitude", "latitude"], inplace=True)
# 4
geography["longitude"] = geography["longitude"].astype(float)

<b>NOTE:</b> Now we will drop duplicates from geography dataset.

In [ ]:
geography.drop_duplicates(subset=["code_insee"], keep="first", inplace=True)

<b>NOTE:</b> Let's define a auxiliary function that will help us calculate the distance between two points on the map.

In [ ]:
def distance(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1 = radians(lon1)
    lat1 = radians(lat1)
    lon2 = radians(lon2)
    lat2 = radians(lat2)
    
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

def distance_bigger_than(limit, lon1, lat1, lon2, lat2):
    dist = distance(lon1, lat1, lon2, lat2)
    if dist > limit:
        return 0
    else:
        return 1

<b>NOTE:</b> Let's create new PARIS_CLOSE feature. 

In [ ]:
# extracting the coordinates of paris
paris_lat = geography.loc[geography['nom_commune'] == "Paris"].iloc[0]["latitude"]
paris_lon = geography.loc[geography['nom_commune'] == "Paris"].iloc[0]["longitude"]

# auxiliary list that will hold calculated values of PARIS_CLOSE
distances = []

# calculating values of PARIS_CLOSE
for index, row in geography.iterrows():
    distances.append(distance_bigger_than(30, row["longitude"], row["latitude"], paris_lon, paris_lat))

# adding new column to DataFrame
geography["PARIS_CLOSE"] = pd.Series(distances, index=geography.index)

<b>NOTE:</b> Let's create new MAJOR_CITY_DISTANCE feature. 

In [ ]:
# extracting the coordinates of capitals of French provinces
majors =  geography[geography["nom_commune"] == geography["chef.lieu_région"]]
major_lats = majors["latitude"].tolist()
major_lons = majors["longitude"].tolist()

# auxiliary list that will hold calculated values of MAJOR_CITY_DISTANCE
distances = []

# calculating values of MAJOR_CITY_DISTANCE
for index, row in geography.iterrows():
    
    single_distances = []
    for lat, lon in zip(major_lats, major_lons):
        single_distances.append(int(distance(row["longitude"], row["latitude"], lon, lat)))
    
    distances.append(min(single_distances))

# adding new column to DataFrame
geography["MAJOR_CITY_DISTANCE"] = pd.Series(distances, index=geography.index)

<b>NOTE:</b> Let's merge salary and geography datasets and check if our assumption about the relationship between the distance from big cities and the size of salaries were correct.

In [ ]:
salary_location = salary.merge(geography, how="left", left_on='CODGEO', right_on="code_insee")

In [ ]:
salaries_by_dep = salary_location.groupby("numéro_département").mean()
departments_map = departments_map.merge(salaries_by_dep, how="left", left_on="code", right_index=True)
departments_map.dropna(subset = ["longitude", "latitude"], inplace=True)

fig, ax = plt.subplots(1, figsize=(15,14))
ax.set_title('Salary by Departments', size=32, x = 0.25, y=0.90)
fig.patch.set_facecolor((202/255, 204/255, 206/255))
departments_map.plot(ax=ax, column="SNHM14", cmap=plt.cm.plasma, scheme='fisher_jenks', k=10, legend=True)
leg = ax.get_legend()
ax.set_axis_off()
leg.set_bbox_to_anchor((0., 0., 0.2, 0.45))
leg.set_title("Mean net salary")

<b>NOTE:</b> The visualization made clear that people living in close proximity to Paris earn on average more than the rest of the French population.

In [ ]:
trace1 = go.Bar(
    x = ["Far [distance > 30km]", "Close [distance < 30km]"],
    y = salary_location.groupby("PARIS_CLOSE")["SNHM14"].mean().tolist(),
    name='Distance from Paris',
    marker=dict(
        color='rgba(55, 128, 191, 0.7)',
        line=dict(
            color='rgba(55, 128, 191, 1.0)',
            width=2,
        )
    )
)

data = [trace1]
layout = go.Layout(
    title = 'Distance from Paris',
    width=850,
    height=500,
    paper_bgcolor='rgb(202, 204, 206)',
    plot_bgcolor='rgb(202, 204, 206)',
    yaxis = dict(
        title= 'Average earnings [€/hour]',
        anchor = 'x',
        rangemode='tozero'
    ),
    xaxis = dict(title= 'Distance from Paris')
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [ ]:
plt.figure(figsize=(16,10))
sns.set_style("whitegrid")
plt.title('Distance from closest major city', fontsize=20, fontweight='bold', y=1.05,)
plt.xlabel('Distance from closest major city [km]', fontsize=15)
plt.ylabel('Average earnings [€/hour]', fontsize=15)

years = salary_location["MAJOR_CITY_DISTANCE"].values
memory = salary_location["SNHM14"].values

plt.scatter(years, memory, edgecolors='black')
plt.show()

### 2.4.3. Industry dataset preprocessing
<a id="industry_dataset_preprocessing"></a>

<b>NOTE:</b> Similarly as in the case of a dataset describing wage, we are going to switch "CODGEO" column type from object (string) to int, but first we need to get rid of incorrect rows. Some of rows contain non numeric characters. We will select only those values that can be transformed into numbers.

In [ ]:
industry = industry[industry["CODGEO"].apply(lambda x: str(x).isdigit())]
industry["CODGEO"] = industry["CODGEO"].astype(int)

<b>NOTE:</b> The criteria for defining the size of a business differ from country to country. According to the European Commission, we can divide enterprises info five categories based on number of employees: Micro < 10, Small < 50, Medium < 250, Large < 1000 and Enterprise > 1000. Let's create new features that will be closer to representing those oficial categories. We will also create columns that will represent percentage of total number of businesses that is represented by each category.

We will ignore 'E14TS0ND' column becouse we don't know enything concrete about those firms.

In [ ]:
industry['MICRO'] = industry['E14TS1'] + industry['E14TS6']
industry['SMALL'] = industry['E14TS10'] + industry['E14TS20']
industry['MEDIUM'] = industry['E14TS50'] + industry['E14TS100']
industry['LARGE'] = industry['E14TS200'] + industry['E14TS500']

industry['SUM'] = industry['E14TS1'] + industry['E14TS6'] + industry['E14TS10'] + industry['E14TS20'] + industry['E14TS50'] + industry['E14TS100'] + industry['E14TS200'] + industry['E14TS500']

### 2.4.4. Merging datasets
<a id="merging_datasets"></a>

In [ ]:
# merging datasets
full_dataset = reformatted_salary.merge(geography, how="left", left_on='CODGEO', right_on="code_insee")
full_dataset = full_dataset.merge(industry, how="left", on='CODGEO')
# deleting incomplete rows
full_dataset.dropna(inplace = True)
# selecting relevant columns
full_dataset = full_dataset[["POSITION", "SEX", "PARIS_CLOSE", "MAJOR_CITY_DISTANCE", "MICRO", "SMALL", "MEDIUM", "LARGE", "SUM", "WAGE"]]

In [ ]:
colormap = plt.cm.viridis
plt.figure(figsize=(15,15))
plt.title('Correlation of Features', y=1.05, size=15)
sns.heatmap(full_dataset.astype(float).corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)
plt.show()

<b>NOTE:</b> The graph above illustrates the relationship between the successive features of the data set. As expected, the age has the greatest impact on the amount of money earned. The correlation between salary and other characteristics is also perceptible.

### 2.4.5. Splitting dataset into training and testset
<a id="splitting_dataset_into_training_and_testset"></a>

<b>NOTE:</b> First, we need to divide the dataset into matrix of features and vector of values.

In [ ]:
X = full_dataset.iloc[:, :-1].values
y = full_dataset.iloc[:, 9].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

<b>NOTE:</b> Finally, we are ready to build a prediction model.

## 3. Prediction
<a id="prediction"></a>

### 3.1. Linear Regression
<a id="linear_regression"></a>

In [ ]:
from sklearn.linear_model import LinearRegression

regressor_lr = LinearRegression()
regressor_lr.fit(X_train, y_train)

# Predicting the Test set results
y_pred_lr = regressor_lr.predict(X_test)

### 3.2. Random Forest Regressor
<a id="random_forest_regressor"></a>

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor_rfr = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor_rfr.fit(X_train, y_train)

# Predicting the Test set results
y_pred_rfr = regressor_rfr.predict(X_test)

### 3.3. Evaluating the regression model
<a id="evaluating_regression_model"></a>

<b>NOTE:</b> Definition of the standard error function.

In [ ]:
def standard_error(y_pred, y_test):
    sum_value = 0
    for p, t in zip(y_pred, y_test):
        sum_value += (p - t)**2
    
    return((sum_value/len(y_test))**(1/2))

In [ ]:
from sklearn.metrics import r2_score

score_lr = r2_score(y_pred_lr, y_test)
print("r2 for linear regression is equal " + str(round(score_lr, 3)))
score_rfr = r2_score(y_pred_rfr, y_test)
print("r2 for random forest regression is equal " + str(round(score_rfr, 3)))

In [ ]:
error_lr = standard_error(y_pred_lr.tolist(), y_test.tolist())
print("standard error for linear regression is equal " + str(round(error_lr, 3)))
error_rfr = standard_error(y_pred_rfr.tolist(), y_test.tolist())
print("standard error for random forest regression is equal " + str(round(error_rfr, 3)))

# To be continued...

<b>NOTE:</b> At this point, the main task is to test new models and look for even better solutions to our problem.